# URPC development

In [1]:
%discover

test-argon                     serial:///dev/cu.usbmodem1413101        
test-esp32                     serial:///dev/cu.usbserial-0160B5B8     
test-samd                      serial:///dev/cu.usbmodem1413201        
test-stm32                     serial:///dev/cu.usbmodem208E307542522  
test-stm32-cop                 serial:///dev/cu.usbserial-0143520E     


In [1]:
%connect test-stm32
%rsync
%softreset

Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542522
Directories match

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [1]:
from urpc import *
start_client(huzzah_uart()) 

dev-nrf: [Errno 2] could not open port /dev/cu.usbmodem1414201: [Errno 2] No such file or directory: '/dev/cu.usbmodem1414201'

In [1]:
import sys
sys_ = import_('sys')

print("urpc version", version_())
print("platforms: host={}  server={}".format(sys.platform, sys_.get('platform')))

urpc version V1
platforms: host=nRF52840  server=esp32


In [1]:
for i in range(5):
    sys_ = import_('sys')

print("before collect:")
for k,v in registry_().items():
    print("   {:8} {}".format(repr(k),v))
    
import gc
gc.collect()

print("after collect:")
for k,v in registry_().items():
    print("   {:8} {}".format(repr(k),v))
    
print("sys_", sys_)

before collect:
   6        <module 'sys'>
   1        <module 'sys'>
   2        <module 'sys'>
   3        <module 'sys'>
   4        <module 'sys'>
   5        <module 'sys'>
after collect:
   6        <module 'sys'>
sys_ <module 'sys'>


In [1]:
# feature "test"

demo = import_('urpc_demo')
print("demo.a", demo.get('a'))
print("demo.upper", demo.upper("this is upper case!"))
obj = demo.Demo("my demo object")
print("obj =", obj)
print("obj.add:", obj.add(3,9))
print("obj.arg_demo:", obj.arg_demo('this is arg1', 3.1415, kw2='custom kw2'))
print("obj.desc:", obj.get('desc'))
obj.set('desc', 'new value')
print("obj.desc:", obj.get('desc'))
print("obj.new_attr:", obj.get('desc'))
obj.set('new_attr', 'newly created attribute')
print("obj.new_attr:", obj.get('new_attr'))
print("obj.dir:", obj.dir_demo())
print("obj.echo", obj.echo(12345))
print("obj.echo", obj.echo({'abc': 3.1415, 'xyz': 'value of xyz'}))

try:
    obj.foo(7)
except RPCError as e:
    print("\nRPCError should be caught and reported:\n{}".format(e))
    
print("Registry:")
for k,v in registry_().items():
    print("  {} {}".format(k, v))

demo.a 5
demo.upper THIS IS UPPER CASE!
obj = <Demo id=1073651056 desc='my demo object'>
obj.add: 3 + 9 = 12
obj.arg_demo: arg1=this is arg1, arg2=3.1415, kw1=kw1 default, kw2=custom kw2
obj.desc: my demo object
obj.desc: new value
obj.new_attr: new value
obj.new_attr: newly created attribute
obj.dir: ('__class__', '__init__', '__module__', '__qualname__', '__str__', '__dict__', 'add', 'description', '_x', 'desc', 'arg_demo', 'dir_demo', 'echo', 'new_attr')
obj.echo 12345
obj.echo {'xyz': 'value of xyz', 'abc': 3.1415}

RPCError should be caught and reported:
Traceback (most recent call last):
  File "/lib/urpc/async_server.py", line 89, in async_serve
  File "/lib/urpc/async_server.py", line 27, in cm
AttributeError: 'Demo' object has no attribute 'foo'

Registry:
  6 <module 'sys'>
  7 <module 'urpc_demo' from '/lib/urpc_demo.py'>
  8 <Demo object at 3ffe9d70>


In [1]:
# object lifetime

import gc, os
gc_ = import_('gc')

gc.collect()
gc_.collect()

for i in range(12):
    if i == 8: 
        print("host collect")
        gc.collect()
    if i == 9:
        print("server collect")
        gc_.collect()
    demo = import_('urpc_demo')
    data = bytes(os.urandom(1000))
    obj = demo.Demo("obj # {:3}".format(repr(data)))
    print("  [{:3}] registry size = {:3d}, mem_free: host = {:6d}  server = {:6d}".format(i, len(registry_()), gc.mem_free(), gc_.mem_free()))
    del obj
    try:
        print("    obj =", obj)
    except NameError:
        pass

  [  0] registry size =   6, mem_free: host = 132928  server =  95040
  [  1] registry size =   8, mem_free: host = 123728  server =  90352
  [  2] registry size =  10, mem_free: host = 114464  server =  85536
  [  3] registry size =  12, mem_free: host = 105248  server =  80640
  [  4] registry size =  14, mem_free: host =  95728  server =  75440
  [  5] registry size =  16, mem_free: host =  86192  server =  70192
  [  6] registry size =  18, mem_free: host =  76272  server =  64576
  [  7] registry size =  20, mem_free: host =  66352  server =  58928
host collect
  [  8] registry size =   5, mem_free: host = 132000  server =  50960
server collect
  [  9] registry size =   7, mem_free: host = 122816  server =  91904
  [ 10] registry size =   9, mem_free: host = 113696  server =  87216
  [ 11] registry size =  11, mem_free: host = 104416  server =  82352


In [1]:
# speed test

import os, gc, iot

N    = 10     # number of tests
LEN  = [ 0, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048 ]

gc_  = import_('gc')
demo = import_('urpc_demo')
obj  = demo.Demo('speed test')

for l in LEN:
    data = bytes(os.urandom(l))
    gc.collect()
    gc_.collect()
    with iot.Chronometer() as c:
        for n in range(N):
            assert data == obj.echo(data)
    dt = c.elapsed_time
    print("[{:4}]  {:10.1f} kB/s  {:10.1f} requests/s".format(l, 2*l*N/dt/1000, N/dt))

[   0]         0.0 kB/s       306.8 requests/s
[   1]         0.6 kB/s       306.0 requests/s
[   2]         0.9 kB/s       216.9 requests/s
[   4]         2.4 kB/s       300.1 requests/s
[   8]         4.8 kB/s       302.8 requests/s
[  16]         9.5 kB/s       296.3 requests/s
[  32]        17.3 kB/s       269.9 requests/s
[  64]        29.2 kB/s       227.9 requests/s
[ 128]        40.9 kB/s       159.6 requests/s
[ 256]        70.9 kB/s       138.6 requests/s
[ 512]        83.1 kB/s        81.1 requests/s
[1024]        88.9 kB/s        43.4 requests/s
[2048]        91.8 kB/s        22.4 requests/s


In [1]:
N = 100
with iot.Chronometer() as c:
    for n in range(N):
        version_()
dt = c.elapsed_time
print("{:10.1f} requests/s".format(N/dt))

     404.4 requests/s


In [1]:
import socket

N = 100
with iot.Chronometer() as c:
    for n in range(N):
        socket.AF_INET
dt = c.elapsed_time
print("{:10.1f} requests/s".format(N/dt))

   28743.9 requests/s


In [1]:
socket_ = import_('socket')

N = 100
with iot.Chronometer() as c:
    for n in range(N):
        socket_.get("AF_INET")
dt = c.elapsed_time
print("{:10.1f} requests/s".format(N/dt))

     226.7 requests/s


In [1]:
demo_ = import_('urpc_demo')
obj = demo_.Demo("speedy")

N = 100
with iot.Chronometer() as c:
    for n in range(N):
        obj.echo(None)
dt = c.elapsed_time
print("{:10.1f} requests/s".format(N/dt))

     325.9 requests/s
